In [2]:
pip install transformers datasets torchaudio


In [1]:
from transformers import ASTFeatureExtractor, ASTModel
import torchaudio
import torch

# Load the AST feature extractor and model from Hugging Face
feature_extractor = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
def get_vec_repr_of_audio(audio_file_path = "/content/drive/MyDrive/test audio/Power_English_Update.mp3"):
  # Load an MP3 audio file using torchaudio
  audio_file = audio_file_path

  waveform, sample_rate = torchaudio.load(audio_file)

  # If the sample rate is not 16kHz, resample the audio
  if sample_rate != 16000:
      waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

  # Extract features from the waveform using the AST feature extractor
  inputs = feature_extractor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000)

  # Forward pass through the AST model to get audio embeddings (vector representations)
  with torch.no_grad():
      outputs = model(**inputs)

  # Extract the last_hidden_state
  last_hidden_state = outputs.last_hidden_state

  # Perform mean pooling to get a fixed-size vector
  # Mean pooling across the sequence_length dimension
  pooled_vector = last_hidden_state.mean(dim=1).mean(dim=0)
  return pooled_vector

In [4]:
vec_repr = get_vec_repr_of_audio()
vec_repr.shape

torch.Size([768])